In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from skforecast.forecaster_autoreg import ForecasterAutoreg
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


ModuleNotFoundError: No module named 'skforecast.forecaster_autoreg'

In [ ]:
# === 1. Embeddings & Timestamps laden ===
X_embed = np.load("data/tfifd_embeddings_weekly_timestamps.npy")  # shape: (n_weeks, embedding_dim)
timestamps = np.load("data/tfifd_embeddings_weekly.npy")  # shape: (n_weeks,)

df_embed = pd.DataFrame(X_embed)
df_embed['week'] = pd.to_datetime(timestamps)

In [ ]:
# === 2. Tesla Kursdaten laden ===
df_stock = pd.read_csv("TSLA_weekly_adjusted.csv")  # Enthält 'date', 'close', ...
df_stock['date'] = pd.to_datetime(df_stock['date'])
df_stock.rename(columns={'date': 'week'}, inplace=True)

In [ ]:
# Nur relevante Spalten (z. B. Schlusskurs)
df_stock = df_stock[['week', 'close']]

In [ ]:
# === 3. Mergen von Embedding + Aktienkurs ===
df_merged = pd.merge(df_embed, df_stock, on='week', how='inner')
df_merged.sort_values('week', inplace=True)

In [ ]:
# === 4. Feature- & Zielmatrix ===
X = df_merged.drop(columns=['week', 'close'])
y = df_merged['close']

In [ ]:
# === 5. Skalierung ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# === 6. Zeitreihenstruktur aufbauen ===
X_df = pd.DataFrame(X_scaled, index=df_merged['week'])
y_series = pd.Series(y.values, index=df_merged['week'])



In [ ]:
# === 7. Modell trainieren ===
forecaster = ForecasterAutoreg(regressor=Ridge(), lags=5)
split_index = -10  # letzte 10 Wochen als Testdaten

forecaster.fit(y=y_series[:split_index], exog=X_df[:split_index])
y_pred = forecaster.predict(steps=10, exog=X_df[split_index:])


In [ ]:
# === 8. Plot ===
plt.figure(figsize=(10, 4))
plt.plot(y_true.index, y_true.values, label='True')
plt.plot(y_true.index, y_pred.values, label='Forecast')
plt.title("Tesla Weekly Close Forecast (Simuliert)")
plt.xlabel("Woche")
plt.ylabel("Kurs")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()